In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU is enabled and ready to use.")
    except RuntimeError as e:
        print(e)


GPU is enabled and ready to use.


In [3]:
train_dir = os.path.join("dataset", "Train")
test_dir = os.path.join("dataset", "Test")


In [4]:
# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Only rescaling for test set
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 23619 images belonging to 2 classes.
Found 6738 images belonging to 2 classes.


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # Assuming two classes: fresh and rot
])


In [6]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [7]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)


Epoch 1/10
739/739 [==============================] - 668s 886ms/step - loss: 0.4734 - accuracy: 0.7845 - val_loss: 0.3781 - val_accuracy: 0.8313
Epoch 2/10
739/739 [==============================] - 352s 477ms/step - loss: 0.3618 - accuracy: 0.8493 - val_loss: 0.3070 - val_accuracy: 0.8654
Epoch 3/10
739/739 [==============================] - 241s 326ms/step - loss: 0.3226 - accuracy: 0.8687 - val_loss: 0.2940 - val_accuracy: 0.8743
Epoch 4/10
739/739 [==============================] - 229s 310ms/step - loss: 0.2826 - accuracy: 0.8819 - val_loss: 0.2458 - val_accuracy: 0.9007
Epoch 5/10
739/739 [==============================] - 224s 303ms/step - loss: 0.2621 - accuracy: 0.8924 - val_loss: 0.3018 - val_accuracy: 0.8691
Epoch 6/10
739/739 [==============================] - 223s 301ms/step - loss: 0.2385 - accuracy: 0.9019 - val_loss: 0.2319 - val_accuracy: 0.9083
Epoch 7/10
739/739 [==============================] - 223s 301ms/step - loss: 0.2243 - accuracy: 0.9091 - val_loss: 0.2014 -

In [8]:
model.save("rotten_fruit_classifier.h5")


In [9]:
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy*100:.2f}%")


211/211 [==============================] - 19s 90ms/step - loss: 0.2055 - accuracy: 0.9151
Test Accuracy: 91.51%
